In [1]:
import numpy as np
import pandas as pd
import json
# import bike_project as bp

In [2]:
def r_w_bike_trips(path, location_name, num_shops, app_token, limit=50000, offset=50000):
    '''
    Reads an API accessible dataset given url path and other url call variables.
    Writes out to a json file

    Parameters:
    -----------
    path (str): url path beginning with https:
    location_name (str): name to assign to the trip counts column
    token (str): app_token
    limit (int): max rows to request
    offset (int): starting point for gathering rows

    Output:
    --------
    json-formatted file named 'location_name.json'
    '''
    df_a = pd.read_json( f'{path}?$limit={limit}&$offset={0}&$$app_token={app_token}')
    df_b = pd.read_json( f'{path}?$limit={limit}&$offset={offset}&$$app_token={app_token}')
    df = df_a.append(df_b)    
    # rename trip_count column to location_name
    df[location_name] = df.iloc[:, -2:].sum(axis=1)   
    
    # calculate date, month, year, dow, commuter (boolean), trip count am peak, trip count other times
    df['short_date'] = pd.DatetimeIndex(df['date']).date
    df['month'] = pd.DatetimeIndex(df['date']).month
    df['year'] = pd.DatetimeIndex(df['date']).year
    df['dow'] = pd.DatetimeIndex(df['date']).dayofweek
    df['hour'] = pd.DatetimeIndex(df['date']).hour
    df['am_commuter'] = df['dow'].isin([0,1,2,3,4]) & (df['hour'].isin([5,6,7,8,9]))
    df[f'{location_name}_am_peak'] = np.where(df['am_commuter']==True, df[location_name], 0)
    df[f'{location_name}_other'] = np.where(df['am_commuter']==False, df[location_name], 0)
    
    # collapse table by date and create sum counts for commuter and (other-(2 x commuter))
    df_by_date = df.groupby(['short_date', 'month', 'year', 'dow']).agg(
                                        {f'{location_name}_am_peak':'sum',
                                         f'{location_name}_other':'sum'
                                          }).reset_index()
    
    # commuters travel 2 ways--remove assumed pm commuter trips from trip count other
    df[f'{location_name}_other'] = df[f'{location_name}_other'] - df[f'{location_name}_am_peak']
    
    # add in count of nearby bike shops
    df_by_date[f'{location_name}_bike_shops'] = num_shops
    df_by_date.to_json(f'data/{location_name}.json', date_format='iso')
    
    #test output to be sure the read operation went ok
    print(df_by_date.head())
    

In [3]:
urls = {
        'Ballard' : 'https://data.seattle.gov/resource/47yq-6ugv.json',
        'Capitol_Hill' : 'https://data.seattle.gov/resource/j4vh-b42a.json',
        'Central_2nd_Ave' : 'https://data.seattle.gov/resource/avwm-i8ym.json',
# no data until 2019        'Central_7th_Ave' : 'https://data.seattle.gov/resource/qfzg-zmyj.json', 
        'West_Seattle' : 'https://data.seattle.gov/resource/mefu-7eau.json',
        'I90_lid' : 'https://data.seattle.gov/resource/u38e-ybnc.json',
        'NE_Seattle' : 'https://data.seattle.gov/resource/2z5v-ecg8.json',
        'Myrtle_Edwards' : 'https://data.seattle.gov/resource/4qej-qvrz.json'
        }

bike_shops = {
        'Ballard' : 3,
        'Capitol_Hill' : 2,
        'Central_2nd_Ave' : 5,
        'Central_7th_Ave' : 6,
        'West_Seattle' : 3,
        'I90_lid' : 5,
        'NE_Seattle' :6 ,
        'Myrtle_Edwards' : 3
}


In [110]:
# This function call reads the raw data and writes one json output file per location. No need to rerun unless
# datatable is insufficient or in error.

limit = 30000
offset = 30000

with open('data/app_token.txt', 'r') as t_file: 
    app_token = t_file.read() 

for k, v in urls.items():
    r_w_bike_trips(v, k, bike_shops[k], app_token, limit, offset)

   short_date  month  year  dow  Ballard_am_peak  Ballard_other  \
0  2014-01-01      1  2014    2             15.0          369.0   
1  2014-01-02      1  2014    3             80.0          375.0   
2  2014-01-03      1  2014    4             86.0          491.0   
3  2014-01-04      1  2014    5              0.0          555.0   
4  2014-01-05      1  2014    6              0.0          488.0   

   Ballard_bike_shops  
0                   3  
1                   3  
2                   3  
3                   3  
4                   3  
   short_date  month  year  dow  Capitol_Hill_am_peak  Capitol_Hill_other  \
0  2014-01-01      1  2014    2                   5.0               115.0   
1  2014-01-02      1  2014    3                  49.0               140.0   
2  2014-01-03      1  2014    4                  45.0               152.0   
3  2014-01-04      1  2014    5                   0.0               161.0   
4  2014-01-05      1  2014    6                   0.0               

In [4]:
count = 1
for k,v in urls.items():
    if count == 1:
        df = pd.read_json(f'data/{k}.json')
    else:
        df_next = pd.read_json(f'data/{k}.json')
        df = df.merge(df_next, how="left", left_on=["short_date", "month", "year", "dow"], right_on=['short_date', "month", "year", "dow"])
    count += 1
df['date'] = pd.DatetimeIndex(df['short_date']).date


In [5]:
df_seattle_weather = pd.read_csv('data/weather.csv', usecols=['DATE',"PRCP","TAVG","TMAX","TMIN"])
df_seattle_weather['date'] = pd.DatetimeIndex(df_seattle_weather["DATE"]).date

In [6]:
# df = df.merge(df_seattle_weather, how="left", left_on=['short_date'], right_on=['DATE'])
df = df.merge(df_seattle_weather, how="left", left_on="date", right_on='date')
df.index = df['date']

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2345 entries, 2014-01-01 to 2020-06-30
Data columns (total 31 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   short_date                  2345 non-null   object 
 1   month                       2345 non-null   int64  
 2   year                        2345 non-null   int64  
 3   dow                         2345 non-null   int64  
 4   Ballard_am_peak             2345 non-null   int64  
 5   Ballard_other               2345 non-null   int64  
 6   Ballard_bike_shops          2345 non-null   int64  
 7   Capitol_Hill_am_peak        2314 non-null   float64
 8   Capitol_Hill_other          2314 non-null   float64
 9   Capitol_Hill_bike_shops     2314 non-null   float64
 10  Central_2nd_Ave_am_peak     1980 non-null   float64
 11  Central_2nd_Ave_other       1980 non-null   float64
 12  Central_2nd_Ave_bike_shops  1980 non-null   float64
 13  West_Seattle_am_peak   

In [8]:
df['am_peak_ttl'] = df['Ballard_am_peak'] + df['Capitol_Hill_am_peak'] + \
df['Central_2nd_Ave_am_peak'] + df['West_Seattle_am_peak'] + \
df['I90_lid_am_peak'] + df['NE_Seattle_am_peak'] +df['Myrtle_Edwards_am_peak']
df['am_peak_ttl'].tail(50)

date
2020-05-12    497.0
2020-05-13    450.0
2020-05-14    354.0
2020-05-15    698.0
2020-05-16      0.0
2020-05-17      0.0
2020-05-18    645.0
2020-05-19    673.0
2020-05-20    544.0
2020-05-21    244.0
2020-05-22    666.0
2020-05-23      0.0
2020-05-24      0.0
2020-05-25    135.0
2020-05-26    481.0
2020-05-27    839.0
2020-05-28    741.0
2020-05-29    901.0
2020-05-30      0.0
2020-05-31      0.0
2020-06-01    607.0
2020-06-02    650.0
2020-06-03    709.0
2020-06-04    630.0
2020-06-05    669.0
2020-06-06      0.0
2020-06-07      0.0
2020-06-08    592.0
2020-06-09    266.0
2020-06-10    594.0
2020-06-11    504.0
2020-06-12    431.0
2020-06-13      0.0
2020-06-14      0.0
2020-06-15    275.0
2020-06-16    566.0
2020-06-17    780.0
2020-06-18    754.0
2020-06-19    826.0
2020-06-20      0.0
2020-06-21      0.0
2020-06-22    683.0
2020-06-23    773.0
2020-06-24    694.0
2020-06-25    828.0
2020-06-26    837.0
2020-06-27      0.0
2020-06-28      0.0
2020-06-29    608.0
2020-06-30    4

In [9]:
v

'https://data.seattle.gov/resource/4qej-qvrz.json'

In [10]:
df['other_ttl'] = df['Ballard_other'] + df['Capitol_Hill_other'] + \
df['Central_2nd_Ave_other'] + df['West_Seattle_other'] + \
df['I90_lid_other'] + df['NE_Seattle_other'] +df['Myrtle_Edwards_other']
df['other_ttl'].tail(50)

date
2020-05-12    2161.0
2020-05-13    3822.0
2020-05-14    3365.0
2020-05-15    5933.0
2020-05-16    2045.0
2020-05-17    7531.0
2020-05-18    3781.0
2020-05-19    3689.0
2020-05-20    1802.0
2020-05-21    1330.0
2020-05-22    3168.0
2020-05-23    6798.0
2020-05-24    7667.0
2020-05-25    1569.0
2020-05-26    3851.0
2020-05-27    5109.0
2020-05-28    4109.0
2020-05-29    5322.0
2020-05-30     962.0
2020-05-31    3002.0
2020-06-01    3689.0
2020-06-02    2365.0
2020-06-03    3602.0
2020-06-04    3601.0
2020-06-05    3586.0
2020-06-06    4871.0
2020-06-07    3736.0
2020-06-08    3074.0
2020-06-09    1051.0
2020-06-10    4622.0
2020-06-11    3047.0
2020-06-12    1580.0
2020-06-13    3004.0
2020-06-14    6993.0
2020-06-15    1476.0
2020-06-16    3734.0
2020-06-17    4563.0
2020-06-18    4450.0
2020-06-19    4232.0
2020-06-20    3819.0
2020-06-21    5872.0
2020-06-22    4346.0
2020-06-23    4038.0
2020-06-24    3554.0
2020-06-25    4466.0
2020-06-26    4075.0
2020-06-27    1604.0
2020-06-

In [11]:
for k,v in bike_shops.items():
    few_am_peak, many_am_peak, few_other, many_other = [], [], [], []
    if v <= 3:
        few_am_peak.append('{}_am_peak'.format(k))
        few_other.append(f'{k}_other')
    else:
        print(f'{k} {v}')
        many_am_peak.append(f'{k}_am_peak')
        many_other.append(f'{k}_other')

print(few_am_peak)
                          

Central_2nd_Ave 5
Central_7th_Ave 6
I90_lid 5
NE_Seattle 6
['Myrtle_Edwards_am_peak']
